# Create dice objects to be used in simple dice game simulations

In [13]:
import numpy as np

class Die:
    '''
    Dice have a number of sides and, unless in motion, always have one face up.  
    
    Cocked dice are ignored, and motion/trajectories are assumed to be a random choice from the number of sides.
    '''
    
    def __init__(self,sides):
        self.sides = sides
        self.face_up = np.random.choice([i+1 for i in range(self.sides)])
        
    def roll(self):
        self.face_up = np.random.choice([i+1 for i in range(self.sides)])
        
        return self.face_up

In [14]:
die = Die(sides=6)

In [3]:
die.face_up

2

In [4]:
die.roll()

5

In [5]:
die.face_up

5

In [13]:
import numpy as np
from collections import Counter

class Dice_Game:
    '''
    This simple multiplayer dice game is the Beebe family's variation of a popular dice game, we just call it "Dice".  
    
    Instructions:
    
    Players determine who goes first, and proceed taking turns in an order (typically clockwise around a table).
    
    Each player, in turn, proceeds by first shaking all six dice, 
    and proceeding along a decision tree of subsequent rolls.
    This tree ends in either a zero score, where a shake results in no dice count for points, 
    or the player "keeps" a number of dice which count for points.
    
    The point value of dice are determined per shake, 
    and point values do not change when mixed with dice kept from previous rolls.
    
    COUNTERS:
    
    One: 100
    Five: 50
    
    THREE-OF-A-KIND:
    Ones: 1000
    Sixes: 600
    Fives: 500
    Fours: 400
    Threes: 300
    Twos: 200
    
    STRAIGHT (1,2,3,4,5,6): 2500
    
    FLUSH (ALL-OF-A-KIND): Instant Win.
    
    On the occasion where all dice rolled are "counters", the dice must all be brought back and shook again.
    The turn proceeds as usual, adding the new shake points to the previous all-counter results.
    
    
    '''
    
    def __init__(self, players, num_dice=6):
        self.player_names = ['Player_{}'.format(name + 1) for name in [j for j in range(players)]]
        self.players = [self.Player(name=player) for player in self.player_names]
        self.dice = [self.Die(sides=6) for die in [i for i in range(num_dice)]]
        self.play_to = 7500
        self.winner = False
        self.turn = 0
        
        
    def start():
        self.score = score
        
    def object_list_to_faces(self,die_list):
        '''
        Function takes list (potentially including lists) of die objects and returns a list (of lists) of the die faces.
        '''

        face_list = [i.face_up if type(i) == Dice_Game.Die else [j.face_up for j in i] for i in die_list]

        return face_list
    
    def shake(self,dice_to_roll,Player):
        print('SHAKING')
        shook_dice = [die.roll() for die in dice_to_roll]
        #print('I,', Player.name, 'shook the dice:', [i.face_up for i in dice_to_roll])
        print('I,', Player.name, 'shook the dice:', self.object_list_to_faces(dice_to_roll))
        return dice_to_roll


        
    def player_take_turn(self,Player):
        Player.turn += 1
        still_to_shake = self.dice.copy()
        #kept = []
        Player.turn_shakes = 0
        #print('length of self.dice', len(self.dice))
        while len(still_to_shake) > 0:
            if Player.turn_shakes == 0:
                print('lets goooo!')
                
                shake = self.shake(still_to_shake,Player)
                Player.turn_shakes += 1
                shake_faces = [i.face_up for i in shake]
                #print('shake:',shake_faces)
                counters, bring_back = self.detect_keepable(shake)
                kept, still_to_shake, nuthin = self.keep_dice(shake, counters)
                print('still to shake faces:', self.object_list_to_faces(still_to_shake))
                #kept.append(to_keep)
                Player.learning_history.extend(kept)
                print('KEPT:',kept)
                score = self.kept_dice_faces_to_score(self.object_list_to_faces(kept))
                print('SCORE:', score)
                print('FIRST JUNCTION')
                print('On turn',Player.turn,'shake',Player.turn_shakes,', I,',Player.name,'kept', self.object_list_to_faces(kept),'and my total kept dice is', self.object_list_to_faces(kept))

                if bring_back:
                    print('bringing back from first junction.............')
                    still_to_shake = self.dice.copy()
                
            #elif kept != [] and Player.turn_shakes > 0:
            elif nuthin == 0:
                print("keep 'em rollin'!")
                print('kept so far:', self.object_list_to_faces(kept))
                
                shake = self.shake(still_to_shake,Player)
                Player.turn_shakes += 1
                shake_faces = [i.face_up for i in shake]
                print('shake:',shake_faces)
                counters, bring_back = self.detect_keepable(shake)
                to_keep, still_to_shake, nuthin = self.keep_dice(shake, counters)
                #print('objects still to shake:',still_to_shake)
                #print('still to shake faces:', [i.face_up for i in still_to_shake])
                print('TO KEEP:', self.object_list_to_faces(to_keep))
                kept.extend(to_keep)
                Player.learning_history.append(kept)
                #print('KEPT:',kept)
                print('SECOND JUNCTION')
                print('On turn',Player.turn,'shake',Player.turn_shakes,', I,',Player.name,'kept', self.object_list_to_faces(to_keep),'and my total kept dice is', self.object_list_to_faces(kept))
                if bring_back:
                    print('bringing back from second junction.............')
                    still_to_shake = self.dice.copy()
                    
            #elif kept == [] and Player.turn_shakes > 0:
            elif nuthin == 1:
                print("you got nuthin:'",shake_faces, ". next player's turn.")
                break
        
        
        
        
        
    def detect_keepable(self, shook_dice):
        
        '''
        Method which sorts through the shook dice and returns a list (of lists) of potentially keepable dice.
        
        For example, if the dice objects shook show [1,2,5,4,4,4] detect_keepable will return: [[4,4,4],1,5]
        
        '''
        print('BEGIN DETECTING KEEPABLES')
        
        BRING_EM_BACK = 0
        
        #print('shook dice:', shook_dice)
        
        dice_faces = self.object_list_to_faces(shook_dice)
        #print('dice faces', dice_faces)
        #print('DOUBLE CHECK FACES:', [i.face_up for i in shook_dice])
        
        three_of_a_kinds = [i for i in Counter(dice_faces) if Counter(dice_faces)[i] >= 3]
        #print('three_of_a_kinds:', three_of_a_kinds)
        
        keepable_three_of_a_kinds = []
        for set_of_three in three_of_a_kinds:
            print('set_of_three',set_of_three)
            toak = []
            for i in shook_dice:
                if i.face_up == set_of_three and len(toak) <= 2:
                    toak.append(i)
            keepable_three_of_a_kinds.append(toak)
            #keepable_three_of_a_kinds.append([i for i in shook_dice if i.face_up == set_of_three])
            #print('appended keepable three of a kind objects:', keepable_three_of_a_kinds)

        keepable = keepable_three_of_a_kinds
        #print('keepable objects after adding three of a kinds:', keepable)
        print('keepable dice faces after adding three of a kinds:', self.object_list_to_faces(keepable))
        
        for i in shook_dice:
            if i.face_up == 1 or i.face_up == 5:
                keepable.append(i)
        #print('keepable objects after adding ones and fives:', keepable)
        print('keepable dice faces after adding ones and fives:', self.object_list_to_faces(keepable))     

        if set(dice_faces) == set([1,2,3,4,5,6]):
            keepable.append(shook_dice)
            #print('keepable objects after adding straight:', keepable)
            print('keepable dice faces after adding straight:', self.object_list_to_faces(keepable))
            
        print('total keepable list of dice faces:', self.object_list_to_faces(keepable))
        
        if all(die in keepable for die in shook_dice):
            print('total keepable list of dice faces:', self.object_list_to_faces(keepable))
            print('ALL COUNTERS.  BRING EM BACK!  NEED FUNCTION HERE!!!!')
            BRING_EM_BACK = 1
        
        return keepable, BRING_EM_BACK
    
    
    def keep_dice(self, shook_dice, keepable_list):
        '''
        Method for choosing which dice to keep.  
        
        This will in particular need to be interfaced for agent to learn.
        
        '''
        
        print('BEGIN KEEPING DICE')
        
        nuthin = 0
        
        #print('keepable object list:', keepable_list)
        #print('keepable dice faces list:', self.object_list_to_faces(keepable_list))
        #print('len keepable_list:', len(keepable_list))
        if len(keepable_list) is 0 or len(keepable_list) is 1:
            kept_dice = keepable_list
        else:
            kept_dice = [np.random.choice(keepable_list)]
        #print('kept dice objects:', kept_dice)
        print('kept dice faces:', self.object_list_to_faces(kept_dice))

        for i in kept_dice:
            #print('object i in kept_dice:', i)
            if type(i) is list:
                print('i in kept_dice is list:', [j.face_up for j in i])
                for j in i:
                    #print('j object in list i:', j)
                    print('j.face_up in list i:', j.face_up)
                    shook_dice.remove(j)
                    #print('shook_dice removed j object:',shook_dice)
                    print('remaining shook_dice faces after removing j:', [i.face_up for i in shook_dice])
            elif i in shook_dice:
                shook_dice.remove(i)

        dice_still_to_shake = shook_dice
        print('dice still to shake faces:', [i.face_up for i in dice_still_to_shake])
        
        if kept_dice == []:
            nuthin = 1

        return kept_dice, dice_still_to_shake, nuthin

    
    def kept_dice_faces_to_score(self,kept_dice_faces):

        score = 0

        if all(isinstance(i,int) for i in kept_dice_faces):
            if set(kept_dice_faces) == set([1,2,3,4,5,6]):
                score += 2500
                print('STRAIGHT', kept_dice_faces, 'SCORES:', score)
        else:
            for i in kept_dice_faces:
                if isinstance(i,list):
                    j = i[0]
                    if j >= 2:
                        score += j*100
                        print('TOAK', i, 'SCORES:', j*100)
                    else:
                        score += 1000
                        print('TOAK of ONES', i, 'SCORES:', 1000)
                else:
                    if i == 1:
                        score += 100
                        print('SINGLE', i, 'SCORES', 100)
                    elif i == 5:
                        score += 50
                        print('SINGLE', i, 'SCORES', 50)

        return score
    
    
    class Die:
        '''
        Dice have a number of sides and, unless in motion, always have one face up.  

        Cocked dice are ignored, and motion/trajectories are assumed to be a random choice from the number of sides.
        '''

        def __init__(self,sides):
            self.sides = sides
            self.face_up = np.random.choice([i+1 for i in range(self.sides)])

        def roll(self):
            self.face_up = np.random.choice([i+1 for i in range(self.sides)])

            return self.face_up
    
    # Players as subclass, called in Dice_Game init
    class Player:
        '''
        Each player is equipped with several attributes, mainly for statistics and for future agent-based learning.
        '''
    
        def __init__(self,name):
            self.name = name
            self.turn = 0
            self.score = 0
            self.learning_history = []
            self.turn_shakes = 0
            self.game_shakes = 0

            


In [14]:
game = Dice_Game(players=3)

In [15]:
game.play_to

7500

In [16]:
print([i.face_up for i in game.dice])

[2, 4, 2, 6, 3, 4]


In [17]:
game.players[0].name

'Player_1'

In [18]:
shake = game.shake(game.dice,game.players[0])

SHAKING
I, Player_1 shook the dice: [1, 5, 6, 6, 1, 1]


In [19]:
game.player_take_turn(game.players[0])

lets goooo!
SHAKING
I, Player_1 shook the dice: [2, 5, 5, 6, 2, 2]
BEGIN DETECTING KEEPABLES
set_of_three 2
keepable dice faces after adding three of a kinds: [[2, 2, 2]]
keepable dice faces after adding ones and fives: [[2, 2, 2], 5, 5]
total keepable list of dice faces: [[2, 2, 2], 5, 5]
BEGIN KEEPING DICE
kept dice faces: [5]
dice still to shake faces: [2, 5, 6, 2, 2]
still to shake faces: [2, 5, 6, 2, 2]
KEPT: [<__main__.Dice_Game.Die object at 0x10a32f210>]
SINGLE 5 SCORES 50
SCORE: 50
FIRST JUNCTION
On turn 1 shake 1 , I, Player_1 kept [5] and my total kept dice is [5]
keep 'em rollin'!
kept so far: [5]
SHAKING
I, Player_1 shook the dice: [3, 1, 1, 6, 4]
shake: [3, 1, 1, 6, 4]
BEGIN DETECTING KEEPABLES
keepable dice faces after adding three of a kinds: []
keepable dice faces after adding ones and fives: [1, 1]
total keepable list of dice faces: [1, 1]
BEGIN KEEPING DICE
kept dice faces: [1]
dice still to shake faces: [3, 1, 6, 4]
TO KEEP: [1]
SECOND JUNCTION
On turn 1 shake 2 , I

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:209: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
game.players[0].turn

In [41]:
def kept_dice_faces_to_score(kept_dice_faces):
    
    score = 0
    
    if all(isinstance(i,int) for i in kept_dice_faces):
        if set(kept_dice_faces) == set([1,2,3,4,5,6]):
            score += 2500
            print('STRAIGHT', kept_dice_faces, 'SCORES:', score)
    else:
        for i in kept_dice_faces:
            if isinstance(i,list):
                j = i[0]
                if j >= 2:
                    score += j*100
                    print('TOAK', i, 'SCORES:', j*100)
                else:
                    score += 1000
                    print('TOAK of ONES', i, 'SCORES:', 1000)
            else:
                if i == 1:
                    score += 100
                    print('SINGLE', i, 'SCORES', 100)
                elif i == 5:
                    score += 50
                    print('SINGLE', i, 'SCORES', 50)
    
    return score

In [146]:
def stay_or_go(potential_score,Player):
    
    
    
    return decision

In [24]:
diex = Dice_Game.Die(6)

In [25]:
type(diex)

__main__.Dice_Game.Die

In [26]:
type(diex) == Dice_Game.Die

True

In [27]:
diex

In [28]:
diex.face_up

4

In [29]:
diey = Dice_Game.Die(6)
diey.face_up

4

In [30]:
diez = Dice_Game.Die(6)
diez.face_up

5

In [31]:
list_of_die = [diex,diey,diez]

In [33]:
list_of_faces = [diex.face_up,diey.face_up,diez.face_up]
list_of_faces

[4, 4, 5]

In [34]:
if diex in list_of_faces:
    print('h')

In [35]:
list_of_faces.extend([3,[3,4]])
list_of_faces

[4, 4, 5, 3, [3, 4]]

In [37]:
all(die.face_up in list_of_faces for die in list_of_die)

True

In [38]:
print(die.face_up in list_of_faces for die in list_of_die)

<generator object <genexpr> at 0x11e63a250>


In [30]:
def object_list_to_faces(die_list):
    '''
    Function takes list (potentially including lists) of die objects and returns a list (of lists) of the die faces.
    '''
    
    face_list = [i.face_up if type(i) == Dice_Game.Die else [j.face_up for j in i] for i in die_list]
    
    return face_list
    

In [43]:
dices = [1,5,[3,3,3]]
kept_dice_faces_to_score(dices)

SINGLE 1 SCORES 100
SINGLE 5 SCORES 50
TOAK [3, 3, 3] SCORES: 300


450

In [39]:
dices

[1, 2, 3, 4, 5, 6]

In [40]:
all(isinstance(i,int) for i in dices)

True

In [22]:
type(list) in dices

False

In [23]:
for i in dices:
    print(type(i))

<class 'int'>
<class 'int'>
<class 'list'>
